In [1]:
from functools import reduce
from datetime import datetime

from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np


# Manual exploration of html tags

In [37]:
base_url = 'https://www.infocasas.com.uy'
# source_url = 'https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/'
source_url = 'https://www.infocasas.com.uy/venta/inmuebles/montevideo/'
url_page = source_url + 'pagina{}'.format(953)
print(url_page)
page = urllib.request.urlopen(url_page)
soup = BeautifulSoup(page, 'html.parser')

table = soup.find_all('div', attrs={'class': 'propiedades-slider'})

https://www.infocasas.com.uy/venta/inmuebles/montevideo/pagina953


In [41]:
a1 = 'https://www.infocasas.com.uy/venta/inmuebles/montevideo/pagina953'
a2 = 'https://www.infocasas.com.uy/venta/inmuebles/montevideo/pagina954'
a2>a1

True

In [40]:
soup.find('div', attrs={'id': 'paginado'}).find('a', attrs={'class': 'next', }).attrs['href']

'/venta/inmuebles/montevideo/pagina954'

In [3]:
neighborhood = [
    [k.text for k in p.find_all('p')] for t in table \
    for p in t.find_all('div')\
    if 'singleLineDots' in p['class']
]
price = [p.text.split()[-1] for t in table \
         for p in t.find_all('div') if 'precio' in p['class']]

desc = [[k.text for k in p.find_all('p')] for t in table \
        for p in t.find_all('div') if 'inDescription' in p['class']]
desc = [k[0] for k in desc]

details = [[d.find_all('span')[0].text for d in p.find_all('div')]\
           for t in table for p in t.find_all('div')\
           if 'contentIcons' in p['class']]
details = pd.DataFrame(details, columns=['rooms', 'bathrooms', 'area_m2'])

data_id = [k.get('data-id', '') for k in table]
data_idproject = [k.get('data-idproyecto', '') for k in table]
link = [base_url + k.find('a')['href'] for k in table]
title = [k.find('a')['title'] for k in table]

proyecto_label = [k.find(class_='proyectoLabel').get_text() if k.find(class_='proyectoLabel') else None for k in table]




df = pd.DataFrame(neighborhood, columns=['neighborhood', 'type'])
df['price'] = price
df['desc'] = desc
df['uris'] = link
df['id'] = data_id
df['idproject'] = data_idproject
df['title'] = title
df['project_label'] = proyecto_label
df = pd.concat([details, df], axis=1)


In [4]:
df

,rooms,bathrooms,area_m2,neighborhood,type,price,desc,uris,id,idproject,title,project_label
0,1,1,32,Sayago,Apartamentos,13.000,SAYAGO:u13.000-(ref:1459)Alquiler apartamentoD...,https://www.infocasas.com.uy/sayagou-13000apar...,186078346,,"Sayago:$u 13.000-apartamento Reciclado,bien Ub...",None
1,3,3+,146,Punta Carretas,Casas,2.850,"92m2 Terreno 146m2 ConstruidosPB: Living, esta...",https://www.infocasas.com.uy/se-alquila-casa-e...,186079064,,Se Alquila Casa En Pocitos Amoblada,None
2,1,1,35,Paso Molino,Apartamentos,17.000,Apartamentos a estrenar en Paso Molino. Por ...,https://www.infocasas.com.uy/apartamento-de-un...,185938017,,Apartamento De Un Dormitorio En Paso Molino,None
3,1,1,40,Pocitos,Apartamentos,22.000,"Mucha luz, apartamento de un dormitorio a pul...",https://www.infocasas.com.uy/piso-alto-apartam...,186067181,,"Piso Alto, Apartamento De 1 Dorm, Despejado.",None
4,2,1,50,Centro,Apartamentos,24.000,Alquiler 2 dormitorios y servicio sobre calle ...,https://www.infocasas.com.uy/alquiler-2-dormit...,185985797,,Alquiler 2 Dormitorios En El Centro,None
5,5+,3+,450,Carrasco,Casas,7.000,"ESTUPENDA CASA LA ZONA MAS LINDA DE CARRASCO, ...",https://www.infocasas.com.uy/importante-reside...,11173,,Importante Residencia,None
6,2,1,50,Cordón,Apartamentos,22.000,CR PROPIEDADES OFRECEHERMOSO APARTAMENTO DE 2 ...,https://www.infocasas.com.uy/2-dorm-a-estrenar...,186081084,,"2 Dorm, A Estrenar. Cordon",None
7,2,1,73,Malvín,Apartamentos,25.000,"E-Tower es un importante complejo urbano, situ...",https://www.infocasas.com.uy/departamento-malv...,186025399,,Departamento - Malvín,None
8,3,1,103,Carrasco Norte,Apartamentos,1.650,"Apartamento en el segundo piso, unidad 206. 10...",https://www.infocasas.com.uy/carrasco-norte/18...,186028711,,Carrasco Norte,None
9,3,3+,132,Carrasco,Apartamentos,2.800,URBAN HERITAGE OFRECE Excelente apartamento en...,https://www.infocasas.com.uy/terrazas-de-santa...,250952,,Terrazas De Santander - Apt 3 Dorm Con Vista A...,None
